In [1]:
import numpy as np
import gym
from tqdm import tqdm
import matplotlib.pyplot as plt

In [4]:
env = gym.make('FrozenLake-v0')

In [5]:
start = env.reset()

In [9]:
env.render(mode='rgb_array')


SFFF
FHFH
FFFH
HFFG


In [22]:
def Policy_Iteration(env,gamma = 0.9, eps = 1.0e-5, maxIterations = 1000):
    #random policy
    policy = np.random.choice(env.env.nA,size=env.env.nS)
    Vs = np.zeros(env.env.nS)
    new_Vs = np.zeros(env.env.nS)
    
    
    for i in range(maxIterations):
        # policy evaluation, evaluation old_policy
        while True:
            for s in range(env.env.nS):
                a = policy[s]
                new_Vs[s] = np.sum([p * (r + gamma * Vs[s_next]) for p,s_next,r,_ in env.env.P[s][a]])
            if np.all(np.fabs(new_Vs - Vs) < eps):
                break
            Vs = np.copy(new_Vs)
        
        # policy improvement
        # get new policy
        new_policy = np.copy(policy)
        for s in range(env.env.nS):
            
            q_as = [r+gamma *Vs[s_next] for _,s_next,r,_ in env.env.P[s][a]]
            new_policy[s] = np.argmax(np.array(q_as))
        
        if np.all(new_policy == policy):
            print("Policy Converged at {} Iteration".format(i))
            break
        
        old_policy = new_policy
    
    return new_policy
        

In [23]:
def evaluate_Policy(env,policy,gamma=0.9,repeats = 100):
    rewards =[]
    for i in tqdm(range(repeats)):
        s = env.reset()
        discount = 1.0
        total_r = 0.0
        while True:
            s,r,done,_ = env.step(int(policy[s]))
            total_r += discount * r
            if done:
                break
            
            discount *=gamma
            
        rewards.append(total_r)
    
    return np.mean(np.array(rewards))

In [24]:
policy = Policy_Iteration(env)
average_reward = evaluate_Policy(env,policy)

100%|██████████| 100/100 [00:00<00:00, 2535.21it/s]


In [25]:
average_reward

0.0

In [21]:
policy

array([0, 0, 0, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0])